In [ ]:
from datasets import load_dataset
dataset = load_dataset(
   'web_nlg', 'webnlg_challenge_2017')

In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['category', 'size', 'eid', 'original_triple_sets', 'modified_triple_sets', 'shape', 'shape_type', 'lex', 'test_category', 'dbpedia_links', 'links'],
        num_rows: 6940
    })
    dev: Dataset({
        features: ['category', 'size', 'eid', 'original_triple_sets', 'modified_triple_sets', 'shape', 'shape_type', 'lex', 'test_category', 'dbpedia_links', 'links'],
        num_rows: 872
    })
    test: Dataset({
        features: ['category', 'size', 'eid', 'original_triple_sets', 'modified_triple_sets', 'shape', 'shape_type', 'lex', 'test_category', 'dbpedia_links', 'links'],
        num_rows: 4615
    })
})

In [4]:
#show me a sample within the dataset
dataset['train'][0]

{'category': 'Airport',
 'size': 1,
 'eid': 'Id1',
 'original_triple_sets': {'otriple_set': [['Aarhus_Airport | cityServed | "Aarhus, Denmark"@en']]},
 'modified_triple_sets': {'mtriple_set': [['Aarhus_Airport | cityServed | "Aarhus, Denmark"']]},
 'shape': '',
 'shape_type': '',
 'lex': {'comment': ['good', 'good'],
  'lid': ['Id1', 'Id2'],
  'text': ['The Aarhus is the airport of Aarhus, Denmark.',
   'Aarhus Airport serves the city of Aarhus, Denmark.'],
  'lang': ['', '']},
 'test_category': '',
 'dbpedia_links': [],
 'links': []}

In [1]:
conda install -c conda-forge sentencepiece


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.3.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.

Retrieving notices: ...working... done

Note: you may need to restart the kernel to use updated packages.


In [4]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset

# define the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')


/Users/davidguzman/miniforge3/envs/pytorch/lib/python3.11/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [16]:
# define the dataset class
class WebNLGDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.prefix = "translate from Graph to Text: "

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        # preprocess the input graph
        try:
            input_text = self.prefix + "<H> " + item['original_triple_sets']['otriple_set'][0][0] + " <R> " + item['original_triple_sets']['otriple_set'][0][1] + " <T> " + item['original_triple_sets']['otriple_set'][0][2]
        except (KeyError, IndexError):
            input_text = self.prefix
        # preprocess the target text
        try:
            target_text = item['lex']['text'][0]
        except (KeyError, IndexError):
            target_text = ""
        # encode the inputs and targets using the tokenizer
        input_ids = tokenizer.encode(input_text, return_tensors='pt', padding='max_length', max_length=128, truncation=True)
        target_ids = tokenizer.encode(target_text, return_tensors='pt', padding='max_length', max_length=128, truncation=True)
        return input_ids.squeeze(0), target_ids.squeeze(0)



In [17]:

# load the WebNLG dataset
dataset = load_dataset('web_nlg', 'webnlg_challenge_2017')['train']
train_data = WebNLGDataset(dataset)

Found cached dataset web_nlg (/Users/davidguzman/.cache/huggingface/datasets/web_nlg/webnlg_challenge_2017/0.0.0/28ffb892f7f42450dd9558684aa43bcaf44b1b3bf0d77cb8d73534646af88dda)
100%|██████████| 3/3 [00:00<00:00, 839.64it/s]


In [18]:
# set up the data loader
batch_size = 16
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

In [22]:
MAX_INPUT_LENGTH = 128
MAX_TARGET_LENGTH = 128
tokenizer.model_max_length = MAX_INPUT_LENGTH
model.config.max_length = MAX_TARGET_LENGTH

# set up the optimizer and the loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
criterion = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)


# set up the device (GPU or CPU)
device = torch.device('mps') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [25]:
# fine-tune the model
num_epochs = 1
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs, labels=targets)
        loss = criterion(outputs.logits.view(-1, outputs.logits.size(-1)), targets.view(-1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_data)
    print(f"Epoch {epoch+1}/{num_epochs} - loss: {epoch_loss:.4f}")


Epoch 1/1 - loss: 2.4327
